### Generation of predictions using Chemical Transformation Simulator (CTS)
- Created by: Louis Groff
- PIs: Imran Shah and Grace Patlewicz (GP)
- Last modified by GP: 27 March 2024
- Changes made: Test sample SMILES to verify functionality of functional calls to produce HCD and CTS outputs

## Running CTS

CTS can be run using its REST API at https://qed.epa.gov/cts/rest. The API is queried via a post call using the requests package, where the url parameter is: 
https://qed.epa.gov/cts/rest/metabolizer/run and the POST with the model parameters (json input keyword in request.post) is given as a dictionary with three parameters, “structure” (SMILES string), “generationLimit” (integer value between 1-4 to indicate desired transformation depth), and “transformationLibraries” (indicates model choice). The ChemAxon Human Phase I metabolizer is chosen by setting “transformationLibraries” to [“mammalian_metabolism”]. Thus the json parameter with the POST dictionary structure is:
{"structure": <qsar_ready_smiles>, "generationLimit": <1-4>, "transformationLibraries": ["mammalian_metabolism"]}
The generationLimit of 3 was chosen because the current CTS REST API (as of November 2023) is prone to running out of memory for large metabolism trees. Workarounds and solutions to this issue are in progress, but it is recommended to limit the transformation depth between 1-3 generations for the greatest number of successfully returned metabolism queries.

### Import relevant libraries

In [1]:
import os, sys

### Set LIB directory to call functions from metsim package

In [2]:
LIB = os.getcwd().replace("notebooks", "")

In [3]:
if not LIB in sys.path: 
    sys.path.insert(0,LIB)

In [4]:
from metsim.sim.metsim_cts import *

### Testing CTS metsim function for sample chemical using 3 generations

In [6]:
test_cts = metsim_run_cts('OCCOCCO',  3)

/home/grace/anaconda3/envs/py39/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qed.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


CTS API successfully returned metabolism predictions for 3 cycles of Phase I metabolism for input SMILES: OCCOCCO


### Processing output of CTS API call to create MetSim format

In [6]:
metsim_cts_phaseI_out(in_smiles = 'OCCOCCO', depth = 1)

/home/grace/anaconda3/envs/py39/lib/python3.9/site-packages/urllib3/connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'qed.epa.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


CTS API successfully returned metabolism predictions for 1 cycles of Phase I metabolism for input SMILES: OCCOCCO
children key found in dictionary
3 children found for precursor generation level 0
precursor-successor relationships appended for current generational level.


{'datetime': '2024-03-28_12h48m59s',
 'software': 'EPA Chemical Transformation Simulator',
 'version': '1.3.2.2',
 'params': {'depth': 1,
  'organism': 'human',
  'site_of_metabolism': False,
  'model': 'ChemAxon Human Phase I'},
 'input': {'smiles': 'OCCOCCO',
  'inchikey': None,
  'casrn': None,
  'hcd_smiles': None,
  'dtxsid': None,
  'chem_name': None},
 'output': [{'precursor': {'smiles': 'OCCOCCO',
    'inchikey': None,
    'casrn': None,
    'hcd_smiles': None,
    'dtxsid': None,
    'chem_name': None,
    'likelihood': 'UNLIKELY'},
   'successors': [{'enzyme': None,
     'mechanism': 'PrimaryAlcoholOxidation',
     'generation': 1,
     'metabolite': {'smiles': 'OCCOCC=O',
      'inchikey': None,
      'casrn': None,
      'hcd_smiles': None,
      'dtxsid': None,
      'chem_name': None,
      'likelihood': 'UNLIKELY'}},
    {'enzyme': None,
     'mechanism': 'O-Dealkylation',
     'generation': 1,
     'metabolite': {'smiles': 'OCCO',
      'inchikey': None,
      'casrn': 

### Testing the HCD function using a QSAR Ready SMILES from the 112 dataset

In [7]:
metsim_hcd_out(smiles = "OCCOCCO")

Attempting query of Cheminformatics Modules Standardizer with SMILES: OCCOCCO...
Query succeeded.


{'smiles': 'OCCOCCO',
 'casrn': '111-46-6',
 'hcd_smiles': 'OCCOCCO',
 'inchikey': 'MTHSVFCYNBDYFN-UHFFFAOYNA-N',
 'dtxsid': 'DTXSID8020462',
 'chem_name': 'Diethylene glycol',
 'likelihood': None}